In [1]:
import numpy as np
import pandas as pd
import random

ROOT = '/home/robert/DataSpellProjects/marchMadness/'

In [2]:
RegularSeason = pd.read_csv(f'{ROOT}/Data/MRegularSeasonDetailedResults.csv')
Tourney = pd.read_csv(f'{ROOT}/Data/MNCAATourneyCompactResults.csv')
TeamIDs = pd.read_csv(f'{ROOT}/Data/MTeams.csv')
Rankings = pd.read_csv(f'{ROOT}/Data/MMasseyOrdinals.csv')

In [3]:
Train = Tourney
Train.drop(['DayNum', 'WLoc', 'NumOT'], axis=1, inplace=True)

In [4]:
def randomize_team(WTeamID):
    if random.randint(0,1):
        return WTeamID
    else:
        return np.NaN

In [5]:
Train['Team1'] = Train['WTeamID'].apply(lambda x: randomize_team(x))
Train['Team1'].fillna(Train.LTeamID, inplace=True)
Train['Team1'] = Train['Team1'].astype(int)
Train['Team2'] = np.where(Train['WTeamID']==Train['Team1'], Train['LTeamID'], Train['WTeamID'])
Train['Team1 Pts'] = np.where(Train['Team1']==Train['WTeamID'], Train['WScore'], Train['LScore'])
Train['Team2 Pts'] = np.where(Train['Team2']==Train['WTeamID'], Train['WScore'], Train['LScore'])
Train.drop(['WScore', 'LTeamID', 'LScore', 'WTeamID'], axis=1, inplace=True)
Train

,Season,Team1,Team2,Team1 Pts,Team2 Pts
0,1985,1116,1234,63,54
1,1985,1120,1345,59,58
2,1985,1250,1207,43,68
3,1985,1229,1425,58,55
4,1985,1325,1242,38,49
...,...,...,...,...,...
2312,2021,1211,1425,85,66
2313,2021,1417,1276,51,49
2314,2021,1124,1222,78,59
2315,2021,1417,1211,90,93


In [6]:
def get_team_rank(team_id, year):
    temp = Rankings[Rankings.TeamID==team_id]
    temp = temp[temp.Season==year]
    return temp['OrdinalRank'].sum() / len(temp.index)
Train['Team1 Rank'] = Train.apply(lambda x: get_team_rank(x['Team1'], x['Season']), axis=1)
Train['Team2 Rank'] = Train.apply(lambda x: get_team_rank(x['Team2'], x['Season']), axis=1)

/tmp/ipykernel_2836/2960857548.py:4: RuntimeWarning: invalid value encountered in long_scalars
  return temp['OrdinalRank'].sum() / len(temp.index)


KeyboardInterrupt: 

In [ ]:
def split_season(team, year):
    win_df = RegularSeason[RegularSeason.WTeamID==team]
    win_df = win_df[win_df.Season==year]
    loss_df = RegularSeason[RegularSeason.LTeamID==team]
    loss_df = loss_df[loss_df.Season==year]
    num_games = len(win_df) + len(loss_df)
    if num_games == 0:
        num_games = 1
    return win_df, loss_df, num_games

In [ ]:
def get_win_perc(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return len(win_df) / num_games
Train['Team1 Win%'] = Train.apply(lambda x:get_win_perc(x['Team1'], x['Season']), axis=1)
Train['Team2 Win%'] = Train.apply(lambda x:get_win_perc(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_points_avg(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WScore'].sum() + loss_df['LScore'].sum())/num_games
Train['Team1 Avg Pts'] = Train.apply(lambda x:get_points_avg(x['Team1'], x['Season']), axis=1)
Train['Team2 Avg Pts'] = Train.apply(lambda x:get_points_avg(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_points_a_avg(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LScore'].sum() + loss_df['WScore'].sum())/num_games
Train['Team1 Avg Pts A'] = Train.apply(lambda x:get_points_a_avg(x['Team1'], x['Season']), axis=1)
Train['Team2 Avg Pts A'] = Train.apply(lambda x:get_points_a_avg(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_fg_perc(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WFGM'].sum() + loss_df['LFGM'].sum()) / (win_df['WFGA'].sum()+loss_df['LFGA'].sum())
Train['Team1 Fg%'] = Train.apply(lambda x:get_fg_perc(x['Team1'], x['Season']), axis=1)
Train['Team2 Fg%'] = Train.apply(lambda x:get_fg_perc(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_fg_perc_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LFGM'].sum() + loss_df['WFGM'].sum()) / (win_df['LFGA'].sum()+loss_df['WFGA'].sum())
Train['Team1 Fg% A'] = Train.apply(lambda x:get_fg_perc_a(x['Team1'], x['Season']), axis=1)
Train['Team2 Fg% A'] = Train.apply(lambda x:get_fg_perc_a(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_three_perc(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WFGM3'].sum() + loss_df['LFGM3'].sum()) / (win_df['WFGA3'].sum()+loss_df['LFGA3'].sum())
Train['Team1 3%'] = Train.apply(lambda x:get_three_perc(x['Team1'], x['Season']), axis=1)
Train['Team2 3%'] = Train.apply(lambda x:get_three_perc(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_three_perc_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LFGM3'].sum() + loss_df['WFGM3'].sum()) / (win_df['LFGA3'].sum()+loss_df['WFGA3'].sum())
Train['Team1 3% A'] = Train.apply(lambda x:get_three_perc_a(x['Team1'], x['Season']), axis=1)
Train['Team2 3% A'] = Train.apply(lambda x:get_three_perc_a(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_freethrows(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return win_df['WFTM'].sum()+loss_df['LFTM'].sum()
Train['Team1 FT'] = Train.apply(lambda x:get_freethrows(x['Team1'], x['Season']), axis=1)
Train['Team2 FT'] = Train.apply(lambda x:get_freethrows(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_freethrows_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return win_df['LFTM'].sum()+loss_df['WFTM'].sum()
Train['Team1 FT A'] = Train.apply(lambda x:get_freethrows_a(x['Team1'], x['Season']), axis=1)
Train['Team2 FT A'] = Train.apply(lambda x:get_freethrows_a(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_or(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WOR'].sum() + loss_df['LOR'].sum()) / num_games
Train['Team1 OR'] = Train.apply(lambda x:get_or(x['Team1'], x['Season']), axis=1)
Train['Team2 OR'] = Train.apply(lambda x:get_or(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_or_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LOR'].sum() + loss_df['WOR'].sum()) / num_games
Train['Team1 OR A'] = Train.apply(lambda x:get_or_a(x['Team1'], x['Season']), axis=1)
Train['Team2 OR A'] = Train.apply(lambda x:get_or_a(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_dr(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WDR'].sum() + loss_df['LDR'].sum()) / num_games
Train['Team1 DR'] = Train.apply(lambda x:get_dr(x['Team1'], x['Season']), axis=1)
Train['Team2 DR'] = Train.apply(lambda x:get_dr(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_dr_a(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LDR'].sum() + loss_df['WDR'].sum()) / num_games
Train['Team1 DR A'] = Train.apply(lambda x:get_dr_a(x['Team1'], x['Season']), axis=1)
Train['Team2 DR A'] = Train.apply(lambda x:get_dr_a(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_to(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['WTO'].sum() + loss_df['LTO'].sum()) / num_games
Train['Team1 TO'] = Train.apply(lambda x:get_to(x['Team1'], x['Season']), axis=1)
Train['Team2 TO'] = Train.apply(lambda x:get_to(x['Team2'], x['Season']), axis=1)

In [ ]:
def get_ta(team, year):
    win_df, loss_df, num_games = split_season(team, year)
    return (win_df['LTO'].sum() + loss_df['WTO'].sum()) / num_games
Train['Team1 TA'] = Train.apply(lambda x:get_ta(x['Team1'], x['Season']), axis=1)
Train['Team2 TA'] = Train.apply(lambda x:get_ta(x['Team2'], x['Season']), axis=1)

In [ ]:
Train = Train[[col for col in Train.columns if col != 'Team1 Pts'] + ['Team1 Pts']]
Train = Train[[col for col in Train.columns if col != 'Team2 Pts'] + ['Team2 Pts']]
Train.dropna(inplace=True)
Train.to_csv(f'{ROOT}/Data/Train.csv')

In [ ]:
Train